In [ ]:
import os
import re
import sys
import glob
import pickle
import tables
import datetime

import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
from tensorflow import keras
from comet_ml.api import API, APIExperiment
from comet_ml.query import Tag

if not '..' in sys.path:
    sys.path.append('..')
from train_LSTM import *

In [ ]:
api = API(api_key = os.environ['COMET_API_KEY'])
workspace = 'danielelinaro'
project_name = 'load-forecasting'

future = 1    # [hours]
history = 24  # [hours]
num_layers = 3
num_neurons = 15

query = Tag('LSTM') & \
        Tag(f'future={future}') & \
        Tag(f'history={history}') & \
        Tag(f'{num_layers}_layers') & \
        Tag(f'{num_neurons}_neurons')

experiments = api.query(workspace, project_name, query, archived=False)
experiment_IDs = []
val_loss = []
hours_ahead = []
for experiment in experiments:
    tags = experiment.get_tags()
    ID = experiment.id
    experiment_IDs.append(ID)
    metrics = experiment.get_metrics()
    val_loss.append(np.array([float(m['metricValue']) for m in metrics 
                              if m['metricName'] == 'val_loss']))
    hours_ahead.append(float([tag for tag in tags if 'ahead' in tag][0][6:]))
    print(f'{ID[:8]} >> hours ahead: {hours_ahead[-1]:4g} ' + 
          f'validation loss: {val_loss[-1].min():.4f}')
hours_ahead = np.array(hours_ahead)
idx = np.argsort(hours_ahead)
hours_ahead = hours_ahead[idx]
val_loss = [val_loss[i] for i in idx]
experiment_IDs = [experiment_IDs[i] for i in idx]

In [ ]:
experiments_path = '../experiments/LSTM/'
n_experiments = len(experiment_IDs)
training_set_min = []
training_set_max = []
models = []
for i,experiment_ID in enumerate(experiment_IDs):
    checkpoint_path = experiments_path + experiment_ID + '/checkpoints/'
    checkpoint_files = glob.glob(checkpoint_path + '*.h5')
    epochs = [int(os.path.split(file)[-1].split('.')[1].split('-')[0])
              for file in checkpoint_files]
    best_checkpoint = checkpoint_files[epochs.index(np.argmin(val_loss[i]) + 1)]
    models.append(keras.models.load_model(best_checkpoint, compile=True))

parameters = pickle.load(open(experiments_path + experiment_ID + '/parameters.pkl', 'rb'))
# we need min and max of the training set to normalize the data
training_set_min = parameters['training_set_min']
training_set_max = parameters['training_set_max']
data_file = '../' + parameters['data_file']
time_step = parameters['time_step']

In [ ]:
data = pickle.load(open(data_file, 'rb'))
df = data['full']['building_energy']
orig_time_step = extract_time_step(df)
df = add_minute_and_workday(df)
df = average_data(df, time_step, orig_time_step, ['consumption', 'generation'])
n_days, samples_per_day = compute_stats(df, time_step)
t = np.arange(samples_per_day) * time_step / 60

print(f'Time step: {time_step} minutes.')
print(f'Number of days: {n_days}.')
print(f'Samples per day: {samples_per_day}.')

In [ ]:
cols = parameters['inputs']['continuous']
if parameters['average_continuous_inputs']:
    cols = [col + '_averaged' for col in cols]
X = make_dataset(df, cols, parameters['inputs']['categorical'],
                training_set_max, training_set_min, n_days, samples_per_day)

In [ ]:
offset = 103 * 96
# today = tf.constant(X[offset : offset + samples_per_day, :][np.newaxis, :, :])
today = X[offset : offset + samples_per_day, :]
tomorrow = np.zeros(samples_per_day)
n_outputs = models[0].outputs[0].shape[1]
for i,model in enumerate(models):
    sys.stdout.write('.')
    sys.stdout.flush()
    tomorrow[i * n_outputs : (i+1) * n_outputs] = model.predict(today)

In [ ]:
fun = lambda y,M,m: m + (M - m) / 2 * (y + 1)
tomorrow_predicted = fun(tomorrow, training_set_max[0], training_set_min[0])
tomorrow_actual = fun(X[offset + samples_per_day : offset + 2 * samples_per_day, 0],
                     training_set_max[0], training_set_min[0])
plt.plot(t, tomorrow_actual, 'k', lw=1, label='Actual')
plt.plot(t, tomorrow_predicted, 'r', lw=1, label='Prediction')
plt.legend(loc='upper right')